In [15]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
# Paths to your training and validation data directories
train_dir = "C:\\Users\\KIIT\\Downloads\\Resume Projects\\Emojify\\Dataset\\train"
val_dir = "C:\\Users\\KIIT\\Downloads\\Resume Projects\\Emojify\\Dataset\\test"

# Create data generators with rescaling (normalize pixel values)
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),         # Resize images to 48x48 (FER2013 format)
    batch_size=64,                # Number of images per batch
    color_mode="grayscale",       # Load images as grayscale
    class_mode='categorical'      # One-hot encode labels
)

# Load and preprocess validation images
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [17]:
# Initialize a sequential model
emotion_model = Sequential()

# Add layers to the model
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))  # 7 output neurons for 7 emotions


In [18]:
emotion_dict = {
    0: "Angry",
    1: "Disgusted",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised"
}


In [19]:
emotion_model.compile(
    loss='categorical_crossentropy',  # Suitable for multi-class classification
    optimizer=Adam(learning_rate=0.0001),  # Adam optimizer with a small learning rate
    metrics=['accuracy']              # Track accuracy during training
)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of batches per epoch
    epochs=50,                                                             # Number of training epochs
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stop]
)

Epoch 1/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 670s 1s/step - accuracy: 0.2464 - loss: 1.8287 - val_accuracy: 0.3583 - val_loss: 1.6778
Epoch 2/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.2969 - loss: 1.7392 - val_accuracy: 0.3557 - val_loss: 1.6790
Epoch 3/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 191s 425ms/step - accuracy: 0.3557 - loss: 1.6430 - val_accuracy: 0.4023 - val_loss: 1.5534
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.3750 - loss: 1.6716 - val_accuracy: 0.4032 - val_loss: 1.5521
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 129s 287ms/step - accuracy: 0.4064 - loss: 1.5300 - val_accuracy: 0.4477 - val_loss: 1.4629
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.4062 - loss: 1.5078 - val_accuracy: 0.4485 - val_loss: 1.4627
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 129s 288ms/step - accuracy: 0.4351 - loss: 1.4698 - val_accuracy: 0.4686 - val_loss: 1.4000
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.4375 - loss: 1.533

In [21]:
# Evaluate the model on the validation set
val_loss, val_accuracy = emotion_model.evaluate(
    validation_generator,
    steps=validation_generator.samples // validation_generator.batch_size
)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

112/112 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - accuracy: 0.6186 - loss: 1.0542
Validation Accuracy: 60.38%


In [25]:
emotion_model.save('model_full.h5')  # Saves architecture + weights

In [23]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    bounding_box = cv2.CascadeClassifier("C:\\Users\\KIIT\\Downloads\\Resume Projects\\Emojify\\myenv\\Lib\\site-packages\\cv2\\data\\haarcascade_frontalface_default.xml")
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1

KeyboardInterrupt: 

In [13]:
import cv2
print(cv2.data.haarcascades)

C:\Users\KIIT\Downloads\Resume Projects\Emojify\myenv\Lib\site-packages\cv2\data\
